In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.insert(0, '/Users/leon/Income/python files/politeness_code')
from model_code.distilbert import *

In [7]:
tokenizer = AutoTokenizer.from_pretrained("../../sentiment_model/sentiment_tokenizer")

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("../../sentiment_model/sentiment_model")
model = disbert_arch(model)
path = '../../sentiment_model/sentiment_model/fine_tune_disbert.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [9]:
# Read csv file
import pandas as pd
df = pd.read_csv("../data_collection/sentiment_data.csv")
df = df.dropna()
df.tail()

,text,score
30,may I put you on hold while I check with my su...,1
31,thank you for holding the line,1
32,may I proceed to help you with the purchase now,1
33,has anyone from us or a broker quoted you,1
34,mr Ang this is how the protection works for you,1


In [10]:
import numpy as np
def softmax(x, axis):
    e_x = np.exp(x - np.max(x, axis))
    return e_x / (e_x.sum(axis) + 1e-9)

In [11]:
def batch_tokenizer(sentence_ls):
    output_ls = []
    for sentence in sentence_ls:
        cur_input = tokenizer(sentence, return_tensors = "pt")
        output = model(cur_input['input_ids'], cur_input['attention_mask']).detach().numpy()[0]
        #soft_output = softmax(output, 0)
        index = np.argmax(output) - 1
        output_ls.append(index)
    return output_ls

In [12]:
df['predicted'] = batch_tokenizer(list(df['text']))

In [13]:
df

,text,score,predicted
0,no problem i will send you the information sho...,1,1
1,maybe i give you a call back. do you prefer to...,0,0
2,okay thank you so much for your time.,1,1
3,then okay i will call again on tuesday bye bye.,0,0
4,sure I can call you back on next tuesday. than...,1,1
5,hi this is leon calling from income ntuc. is i...,1,1
6,ya hello good afternoon speak to leon please. ...,0,0
7,ya you see even just now he message me he said...,-1,-1
8,currently we partnership with abcd. can i chec...,1,1
9,good evening may i speak to mister leon please...,1,1


In [14]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(df['score'], df['predicted']))

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00         3
           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13

    accuracy                           1.00        35
   macro avg       1.00      1.00      1.00        35
weighted avg       1.00      1.00      1.00        35



In [15]:
# Check sentences that label as negative
negative_sen = []
for i in df[df['predicted'] == -1]['text']:
    negative_sen.append(i)

In [16]:
# # Load packages
# import nltk
# import spacy
# import sys
# sys.path.insert(0, "/Users/leon/Income/python files/politeness_code")
# from helper.politeness_helper import *
# # Load spacy nlp model
# nlp = spacy.load('en_core_web_lg')

In [17]:
# # Load standardizer
# with open('../other_collection/standardizer.pkl', 'rb') as f:
#     standardizer = pickle.load(f)    
# # Load custom model/data
# xgb_model = pickle.load(open('../model_collection/final_xgb_v2.sav', 'rb'))

In [18]:
# # Try the politeness model on these negative sentences
# for sen in negative_sen:
#     cur_parse = format_doc(sen, nlp)
#     cur_score = customize_score(cur_parse, xgb_model, standardizer, "prob")
#     print(sen)
#     print(cur_score)